In [ ]:
# Setting options for the plots
%matplotlib inline
%config InlineBackend.figure_formats={'retina', 'svg'}
%config InlineBackend.rc={'savefig.dpi': 150}

In [ ]:
## filter out warnings from third-party libraries to prevent them
# from showing up in the notebooks in multiple places
import warnings

# warning from shap about tqdm 
from tqdm import TqdmExperimentalWarning
warnings.filterwarnings("ignore", 
                        category=TqdmExperimentalWarning, 
                        message="Using `tqdm.autonotebook.tqdm` .*", 
                        module="shap.explainers._linear")

# warning from matplotlib -> seaborn when figure layouts are changed to "tight"
warnings.filterwarnings("ignore", 
                        category=UserWarning, 
                        message=".*figure layout.*", 
                        module="seaborn.axisgrid")# warning from seaborn about a deprecated method in pandas
warnings.filterwarnings("ignore", 
                        category=FutureWarning, 
                        message=".*is_categorical_dtype is deprecated.*", 
                        module="seaborn")



In [ ]:
import itertools
import os
import pickle
import platform
import time
import warnings

from functools import partial
from os.path import abspath, relpath, exists, join

import matplotlib
import numpy as np
import seaborn as sns
import shap
import pandas as pd
from IPython import sys_info
from IPython.display import display, HTML, Image, Javascript, Markdown, SVG
from matplotlib import pyplot as plt

from rsmtool.reader import DataReader
from rsmtool.writer import DataWriter
from rsmtool.utils.files import parse_json_with_comments
from rsmtool.utils.notebook import (float_format_func,
                                    int_or_float_format_func,
                                    compute_subgroup_plot_params,
                                    bold_highlighter,
                                    color_highlighter,
                                    show_thumbnail)
from rsmtool.version import VERSION as rsmtool_version


sns.set_context('notebook')

In [ ]:
rsm_report_dir = os.environ.get('RSM_REPORT_DIR', None)

if rsm_report_dir is None:
    rsm_report_dir = os.getcwd()

rsm_environ_config = join(rsm_report_dir, '.environ.json')
if not exists(rsm_environ_config):
    raise FileNotFoundError('The file {} cannot be located. '
                            'Please make sure that either (1) '
                            'you have set the correct directory with the `RSM_REPORT_DIR` '
                            'environment variable, or (2) that your `.environ.json` '
                            'file is in the same directory as your notebook.'.format(rsm_environ_config))
    
environ_config = parse_json_with_comments(rsm_environ_config)

<style type="text/css">
  div.prompt.output_prompt { 
    color: white; 
  }
  
  span.highlight_color {
    color: red;
  }
  
  span.highlight_bold {
    font-weight: bold;  
  }
    
  @media print {
    @page {
      size: landscape;
      margin: 0cm 0cm 0cm 0cm;
    }

    * {
      margin: 0px;
      padding: 0px;
    }

    #toc {
      display: none;
    }

    span.highlight_color, span.highlight_bold {
        font-weight: bolder;
        text-decoration: underline;
    }

    div.prompt.output_prompt {
      display: none;
    }
    
    h3#Python-packages, div#packages {
      display: none;
  }
</style>

In [ ]:
experiment_id = environ_config.get('EXPERIMENT_ID')
description = environ_config.get('DESCRIPTION')
explanation_path = environ_config.get('EXPLANATION')
background_kmeans_size = environ_config.get('BACKGROUND_KMEANS_SIZE')
id_path = environ_config.get('IDs')
csv_path = environ_config.get('CSV_DIR')
fig_path = environ_config.get('FIG_DIR')
num_display_features = environ_config.get('NUM_FEATURES_TO_DISPLAY')
has_single_example = environ_config.get('HAS_SINGLE_EXAMPLE')

# here we load some objects that we need for our plots
with open(explanation_path, 'rb') as pickle_in:
    explanations = pickle.load(pickle_in, encoding='bytes')
with open(id_path, 'rb') as pickle_in:
    ids = pickle.load(pickle_in, encoding='bytes')

# load the absolute SHAP values from disk
df_abs_shap = DataReader.read_from_file(f"{csv_path}/{experiment_id}_absolute_shap_values.csv", index_col=0)
df_abs_shap.sort_values(by="abs. mean shap", ascending=False, inplace=True)

# javascript path
javascript_path = environ_config.get("JAVASCRIPT_PATH")

# Explanation Report

In [ ]:
# initialize counter for thumbnail IDs
id_generator = itertools.count(1)

In [ ]:
with open(join(javascript_path, "sort.js"), "r", encoding="utf-8") as sortf:
    display(Javascript(data=sortf.read()))

In [ ]:
Markdown('''This report presents the Shap explanations for **{}**: {}'''.format(experiment_id, description))

In [ ]:
HTML(time.strftime('%c'))

In [ ]:
%%html
<div id="toc"></div>